# Selección, entrenamiento y evaluación de un modelo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector

/tmp/ipykernel_20594/3468578804.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/ContenidosPorTemas'
    print('google.colab')

In [3]:
np.random.seed(42)

# Cargamos los datos
df_housing = pd.read_csv("./1_datos/housing.csv")

# Creamos nuestro atributo categórico para los ingresos
df_housing["income_cat"] = pd.cut(df_housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

# Dividimos los datos en conjunto de entrenamiento y prueba
strat_train_set, strat_test_set = train_test_split(df_housing, test_size=0.2, stratify=df_housing["income_cat"], random_state=42)

# Eliminamos la categoria income_cat de ambos conjuntos porque no la usamos
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Separamos predictores y etiquetas
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [4]:
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))


log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())


preprocessing = ColumnTransformer(
    [
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=num_pipeline

    )

## Selección y Entrenamiento

¡Finalmente! Hemos resuelto el problema, obtuvimos los datos, los exploramos, los dividimos en un conjunto de entrenamiento y un conjunto de prueba, y creamos un _pipeline_ de preprocesamiento para limpiar y preparar automáticamente los datos para los algoritmos de aprendizaje maquinal. Ahora estamos listos para seleccionar y entrenar un modelo.

In [5]:
from sklearn.linear_model import LinearRegression

lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('log',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                                       func=<ufunc 'log'>)),
                                                                  ('standa...
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fdd2a4b33a0>)])),
                ('linearregression', LinearRegression())])

In [6]:
housing_predictions = lin_reg.predict(housing)
housing_predictions[:5].round()

array([ 77481., 295588., 180858., 203538., 260212.])

In [7]:
housing_labels.iloc[:5].values

array([ 72100., 279600.,  82700., 112500., 238300.])

Vamos a usar la raiz del error cuadrático medio (RMSE) como medida de desempeño

In [8]:
from sklearn.metrics import mean_squared_error

lin_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
lin_rmse

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


71834.39725072501

El valor de `median_house_value` de los distritos oscila entre \$120,000 y \$265,000, por lo que un error de predicción de \$71,834 no es muy bueno. Este es un ejemplo de un modelo mal ajustado (`underfitting`) a los datos de entrenamiento. Cuando esto ocurre, puede significar que las características no proporcionan suficiente información para realizar buenas predicciones o que el modelo no es lo suficientemente bueno.

Formas de corregir el desajuste son seleccionar un modelo más potente, proporcionar al algoritmo mejores características o reducir las restricciones en el modelo.

In [9]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('impute',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardize',
                                                              StandardScaler())]),
                                   transformers=[('log',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                                       func=<ufunc 'log'>)),
                                                                  ('standa...
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fdd2a4b33a0>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [10]:
housing_predictions = tree_reg.predict(housing)
tree_rmse = mean_squared_error(housing_labels, housing_predictions,squared=False)
tree_rmse

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.0

¿Ningún error en absoluto? Es probable que el modelo se haya sobreajustado (`overfitting`) a los datos. La situación más probable es que el modelo haya memorizado los datos de entrenamiento en lugar de aprender patrones generales que puedan aplicarse a nuevos datos. El sobreajuste ocurre cuando un modelo se ajusta demasiado a los detalles específicos de los datos de entrenamiento y, como resultado, tiene un rendimiento deficiente en datos nuevos y no vistos.

¿Cómo podemos comprobarlo? ¿Usamos los datos de prueba? Como dijimos anteriormente, no miramos el conjunto de prueba ni lo usamos, hasta que un modelo entrenado esté listo para usar, por lo que necesitas utilizar parte del conjunto de entrenamiento para el entrenamiento y otra parte para la validación del modelo.

## Validación del Modelo

Usamos validación cruzada

In [15]:
from sklearn.model_selection import cross_val_score

tree_rmses = -cross_val_score(tree_reg, housing, housing_labels, scoring="neg_root_mean_squared_error", cv=10)
tree_rmses

array([70618.93509952, 68717.20112483, 64811.42966377, 69104.50405262,
       66639.90261572, 67669.9360713 , 72506.47721299, 68698.86759177,
       66948.71049264, 71411.81985203])

In [12]:
pd.Series(tree_rmses).describe()

count       10.000000
mean     68712.778378
std       2337.635618
min      64811.429664
25%      67129.016887
50%      68708.034358
75%      70240.327338
max      72506.477213
dtype: float64

In [13]:
lin_rmses = -cross_val_score(lin_reg, housing, housing_labels,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(lin_rmses).describe()

count       10.000000
mean     71869.661152
std       2757.794518
min      67900.391991
25%      69402.854126
50%      73012.508295
75%      73611.526050
max      75402.422142
dtype: float64

In [14]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
forest_rmses = -cross_val_score(forest_reg, housing, housing_labels, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count       10.000000
mean     49411.910330
std       2263.493135
min      46001.790797
25%      47774.189296
50%      49074.772680
75%      50766.216971
max      53377.797449
dtype: float64

## Ajuste de hiperparámetros del modelo

Usaremos GridSearch para ajustar los parámetros de nuestros modelos (Lo veremos en las siguientes clases)

## Evaluación con los datos de prueba

In [17]:
final_model = forest_reg

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

final_model.fit(housing,housing_labels)

final_predictions = final_model.predict(X_test)

final_rmse = mean_squared_error(y_test, final_predictions, squared=False)
print(final_rmse)

47030.89583130664


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
